<a href="https://colab.research.google.com/github/cliche-niche/model-zoo-submissions/blob/main/RepVGG/RepVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An implementation of [RepVGG](https://arxiv.org/pdf/2101.03697.pdf), A0 architecture. It tests upon CIFAR-10 dataset.

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers

Loading the data, subtracting the mean, and applying horizontal flip augmentation.

In [5]:
(trainIm, trainLab), (testIm, testLab) = tf.keras.datasets.cifar10.load_data()
trainIm = trainIm / 255.0
testIm = testIm / 255.0

trainImMean = np.mean(trainIm, axis=0)
trainIm -= trainImMean
testIm -= trainImMean

trainLab = tf.keras.utils.to_categorical(trainLab, 10)
testLab = tf.keras.utils.to_categorical(testLab, 10)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
datagen.fit(trainIm)

Code of a single stage.

The first layer of a stage down-samples the input using 3x3 convolutions (stride=2, padding=1) and 1x1 convolutions (stride=2, no padding), adds them and applies ReLU activation on their sum.

The rest of the layers use 3x3 conv (stride=1, padding=1), 1x1 conv (stride=1, no padding). Then the output of these layers is added along with an identity, and ReLU activation is applied.

At the end of a stage, BatchNormalization is applied.

In [6]:
class stage(layers.Layer):
    def __init__(self, filters, layer):
        super(stage, self).__init__()
        self.dow3 = layers.Conv2D(filters, kernel_size=3, strides=2, padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.dow1 = layers.Conv2D(filters, kernel_size=1, strides=2, kernel_regularizer=regularizers.l2(1e-4))
        self.con3 = layers.Conv2D(filters, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.con1 = layers.Conv2D(filters, kernel_size=1, padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.bn = layers.BatchNormalization()
        self.re = layers.Activation(activations.relu)
        self.lay = layer

    def call(self, inp):
        x = self.dow3(inp)
        y = self.dow1(inp)
        x = x + y
        x = self.re(x)
        for i in range(self.lay-1):
            y = self.con3(x)
            z = self.con1(x)
            x = x + y + z
            x = self.re(x)
        x = self.bn(x)
        return x

    # def model(self):
    #     x = layers.Input(shape=(224, 224, 3))
    #     return tf.keras.Model(inputs=[x], outputs=self.call(x))


An implementation of RepVGG-A0 model. 5 stages are used with widths `[min(64, 64a), 64a, 128a, 256a, 512b]` and number of layers `[1, 2, 4, 14, 1]` respectively, and `a=0.75` and `b=2.5`. The 5 stages are then followed by GlobalAveragePooling layer and a fully connected layer with 10 channels (for CIFAR-10).

In [7]:
class repvgg(tf.keras.Model):
    def __init__(self, a, b, l=[1,2,4,14,1]):
        super(repvgg, self).__init__()
        self.st1 = stage(min(64, int(64*a)), 1)
        self.st2 = stage(64*a, 2)
        self.st3 = stage(128*a, 4)
        self.st4 = stage(256*a, 14)
        self.st5 = stage(512*b, 1)
        self.gap = layers.GlobalAveragePooling2D()
        self.end = layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(1e-4))
        

    def call(self, inp):
        x = self.st1(inp)
        x = self.st2(x)
        x = self.st3(x)
        x = self.st4(x)
        x = self.st5(x)
        x = self.gap(x)
        x = self.end(x)

        return x


    def model(self, train):
        x = layers.Input(shape=train[0].shape)
        return tf.keras.Model(inputs=x, outputs=self.call(x))

In [8]:
model = repvgg(a=0.75, b=2.5)
#print(model.model().summary())
model.compile(  optimizer = tf.keras.optimizers.Adam(),#SGD(momentum=0.9, learning_rate=0.1),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

In [9]:
#For callbacks
class mcb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
cb = mcb()

In [10]:
model.fit(datagen.flow(trainIm, trainLab, batch_size=len(trainIm[0])), epochs=120, validation_data=(testIm, testLab), callbacks=[cb])

Epoch 1/120
1563/1563 [==============================] - 84s 51ms/step - loss: 2.1773 - accuracy: 0.2388 - val_loss: 2.1738 - val_accuracy: 0.2738
Epoch 2/120
1563/1563 [==============================] - 79s 51ms/step - loss: 2.0616 - accuracy: 0.2774 - val_loss: 2.1260 - val_accuracy: 0.3037
Epoch 3/120
1563/1563 [==============================] - 78s 50ms/step - loss: 1.9820 - accuracy: 0.3020 - val_loss: 2.0651 - val_accuracy: 0.2721
Epoch 4/120
1563/1563 [==============================] - 78s 50ms/step - loss: 1.9304 - accuracy: 0.3115 - val_loss: 1.9819 - val_accuracy: 0.2829
Epoch 5/120
1563/1563 [==============================] - 78s 50ms/step - loss: 1.8151 - accuracy: 0.3513 - val_loss: 1.9451 - val_accuracy: 0.2983
Epoch 6/120
1563/1563 [==============================] - 78s 50ms/step - loss: 1.7695 - accuracy: 0.3656 - val_loss: 1.7004 - val_accuracy: 0.3301
Epoch 7/120
1563/1563 [==============================] - 78s 50ms/step - loss: 1.7468 - accuracy: 0.3822 - val_loss: 1

Achieves an accuracy of **78.03%** on CIFAR-10 test dataset, with 89.81% accuracy on training dataset.

In [12]:
model.evaluate(testIm, testLab)

313/313 [==============================] - 5s 15ms/step - loss: 1.0403 - accuracy: 0.7803


[1.0403088331222534, 0.7803000211715698]

In [14]:
print(model.model(trainIm).summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
stage (stage)                (None, 16, 16, 48)        1728      
_________________________________________________________________
stage_1 (stage)              (None, 8, 8, 48)          46464     
_________________________________________________________________
stage_2 (stage)              (None, 4, 4, 96)          139008    
_________________________________________________________________
stage_3 (stage)              (None, 2, 2, 192)         554496    
_________________________________________________________________
stage_4 (stage)              (None, 1, 1, 1280)        2465280   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0     